In [1]:
%run Utils.ipynb
from sage.rings.finite_rings.integer_mod import IntegerMod_gmp

In [10]:
class Verifier_first_stage():
    
    def __init__(self, Y:IntegerMod_gmp , Y_index : int, excecution_trace_length: int, num_of_queries: int):
        self.domain = get_extented_domain(excecution_trace_length=excecution_trace_length, num_of_queries=num_of_queries)
        self.domian_size = excecution_trace_length*num_of_queries
        self.Y_index = Y_index
        self.Y = Y
    
    def verify(self, proof_stage_one: dict):
            index = self.get_index(proof_stage_one=proof_stage_one)
            self.validate_samples(proof_stage_one=proof_stage_one, index=index)
            self.verifiy_path_and_values(proof_stage_one=proof_stage_one, index=index)
            coeffs = self.get_coefficients(proof_stage_one=proof_stage_one)
            self.validate_cp_value_in_index(proof_stage_one=proof_stage_one, coeffs=coeffs)

    def verifiy_path_and_values(self, proof_stage_one: dict, index: int):
        names = ['p1', 'p2', 'p3', 'cp']
        for name in names:
            internal_proof_dict = proof_stage_one[name]
            root = internal_proof_dict['root']
            value_at_index = internal_proof_dict['value']
            path = internal_proof_dict['path']
            verify_path_by_index(
                root=root, expected_value=value_at_index, index=index, domain_size=self.domian_size, path=path
            )
#             verify_path_by_index(root=root, expected_value=value_at_index, path=path)
            
    def get_index(self, proof_stage_one: dict):
        #print(f"{proof_stage_one['cp']=}")
        cp_tree_root = proof_stage_one['cp']['root']
        return (fiat_shamir_random(cp_tree_root)%self.domian_size)
    
    def get_coefficients(self, proof_stage_one:dict) -> dict:
        names = ['p1', 'p2', 'p3']
        return {name:fiat_shamir_random(proof_stage_one[name]['root']) for name in names}
    
    def validate_cp_value_in_index(self, proof_stage_one: dict, coeffs: dict):
        names = ['p1', 'p2', 'p3']
        expected_value = 0
        for name in names:
            expected_value_int = ((proof_stage_one[name]['value'])[1]*coeffs[name])
            expected_value += expected_value_int%PRIME            
        assert proof_stage_one['cp']['value'][1] == expected_value
    
    def validate_samples(self, proof_stage_one: dict, index: int):
        #Create a copy of poly(x), poly(g*x), poly(g**2 *x)
        exceqution_polinomial_samples=proof_stage_one['ex_poly_samples']
        p_x = exceqution_polinomial_samples['ex_p()']
        p_x_g = exceqution_polinomial_samples['ex_p(g)']
        p_x_g_2 = exceqution_polinomial_samples['ex_p(g**2)']
        
        # Validate_p1_value:
        (x1, y1) = (proof_stage_one['p1']['value'])
        assert self.domain[index] == x1
        assert (p_x-1)._divide_if_possible(x1-gen1024**0) == y1
        # p1 = (poly-1)._divide_if_possible(x-gen**0)
        
        # Validate_p2_value:
        (x2, y2) = (proof_stage_one['p2']['value'])
        assert self.domain[index] == x2
        assert (p_x-self.Y)._divide_if_possible(x2-gen1024**self.Y_index) == y2
        # p2 = (poly - Y)._divide_if_possible(x-gen**index_y)
        
        # Validate_p2_value:
        (x2, y2) = (proof_stage_one['p2']['value'])
        assert self.domain[index] == x2
        assert (p_x-self.Y)._divide_if_possible(x2-gen1024**self.Y_index) == y2
        # p2 = (poly - Y)._divide_if_possible(x-gen**index_y)
        
        
        # Validate_p3_value:
        (x3, y3) = (proof_stage_one['p3']['value'])
        assert self.domain[index] == x3
        n=1024 #trance_len
        constrain_3_numer = p_x_g_2 - p_x_g - p_x
        constrain_3_denom = (x3**n-1)._divide_if_possible( (x3-gen1024**(n-1)) * (x3-gen1024**(n-2)))
        assert y3 == constrain_3_numer._divide_if_possible(constrain_3_denom)
        
        
        
        # names = ['p1', 'p2', 'p3']
        # for name in names:
        #     expected_value_int = ((proof_stage_one[name]['value'])[1]*coeffs[name])
        #     expected_value += expected_value_int%PRIME            
        # assert proof_stage_one['cp']['value'][1] == expected_value
#////////////////////////////////////////////////////////#
#     def verify_path_by_index(self, root: str, expected_value: tuple, index: int, domain_size: int, path: dict):

#         index_size = int(log(domain_size, 2))
#         index_as_str = format(index, f'#0{index_size+2}b')
#         # shift from '0b1110' to '1110'
#         index_as_str = index_as_str[2:] 
#         key=root
#         while(index_as_str):
#             value = path[key]

#             #Verify the hash:
#             assert hash_tow_elements(value[0], value[1]) == key

#             direction_bit = int(index_as_str[0])
#             key = value[direction_bit]
#             index_as_str = index_as_str[1:]

#         #Now the key is the hash of the required index. Reauired value = tree[key] = (coset, CP[coset])
#         value = path[key]
#         assert hash_one_elements(value) == key
#         assert value == expected_value

In [6]:
a:IntegerMod_gmp=14
a.__invert__()

1/14

In [6]:
def verify_second_stage_canceld(proof_stage_two: dict, root_stage_one: str, cp_degree: int):
    #proof structure: {stage: {'root': merkle_root, 'pathes':[pathes],'random':random number for naxt-stage}}
    #path in pathes structure: ((point),{path})

    for stage,proof in proof_stage_two.items():
        last_stage = stage
        
        #Verify pathes:
        current_stage_point = []
        for path in proof['pathes']:
            verify_path_by_index(proof['root'], expected_value = path[0], path= path[1])
            if path[0] not in current_stage_point:
                current_stage_point.append(path[0])
                
        #Verify random:
        assert proof['random'] == fiat_shamir_random(proof['root'])
        next_stage_points = []
        
        #compute next stage points for the first stage:
        if stage == 0:
            #Verify that the original function of stage two is cp from stage on)
            assert proof['root'] == root_stage_one
            for i in range(0,len(current_stage_point),2):
                a,f_a = current_stage_point[i]
                neg_a, neg_f_a = current_stage_point[i+1]
                assert neg_a**2== a**2 #Verify that two conssecutive values are inverse elements of each other
                y = f_a + (f_a - neg_f_a)/(2*a)*(proof['random']-a)
                next_stage_points.append((a**2,y))
                
        #compute next stage points in the general case:
        else:
            for point_a, point_neg_a in zip (current_stage_point, calculated_next_stage_points):
                a,f_a = point_a
                neg_a, neg_f_a = point_neg_a
                assert neg_a**2== a**2 #Verify that two conssecutive values are inverse elements of each other
                y = f_a + (f_a - neg_f_a)/(2*a)*(proof['random']-a)
                next_stage_points.append((a**2,y))
                
        calculated_next_stage_points = next_stage_points
    
    #veify stages number:
    assert last_stage == log(cp_degree,2)-1
    
    #veify last FRI stage is constant:
    for i in range(len(calculated_next_stage_points)-1):
        assert calculated_next_stage_points[i][1] == calculated_next_stage_points[i+1][1]
        
    return True

In [2]:
def verify_second_stage(proof_stage_two: dict, root_stage_zero: str, degree: int, queries: int, field_gen = field_gen): 
    domain_len = queries * degree
    roots = proof_stage_two['root']
    indexes = get_random_index_to_sample(queries, roots, domain_len)
    stages = log(degree, 2)
    pathes = proof_stage_two['pathes']
    #value at index = pathes[stage][index][0]
    #path of value at index = pathes[stage][index][1]
    
    #veify stages number:
    assert stages == len(roots)
    
    for stage in range(stages):
        #verify all point's pathes according to the root
        for index in indexes:
            expected_value = pathes[stage][index][0]
            path = pathes[stage][index][1]
            verify_path_by_index(roots[stage], expected_value, index , domain_len, path)
        #verify first stage points and calculate next stage points
        current_stage_points = [pathes[stage][index][0] for index in indexes]
        rand = fiat_shamir_random(roots[stage])
        if stage==0:
            assert roots[stage] == root_stage_zero
            verify_index_x_value(current_stage_points, indexes, domain_len, field_gen)
            next_stage_points = compute_first_layer(current_stage_points, rand)
        else:
            next_stage_points = compute_next_layer(current_stage_points, calculated_points, rand)
        
        calculated_points =  next_stage_points
        if stage + 1 in range(stages):
            indexes = get_next_stage_indexes(indexes)
        domain_len /= 2
    #verify that the last layer is constant
    for i in range(len(calculated_points)-1):
        assert calculated_points[i][1] == calculated_points[i+1][1]
    
    return True

In [ ]:
def compute_first_layer(current_stage_point, rand):
    next_stage_points =[]
    for i in range(0,len(current_stage_point),2):
        a,f_a = current_stage_point[i]
        neg_a, neg_f_a = current_stage_point[i+1]
        y = f_a + (f_a - neg_f_a)/(2*a)*(rand-a)
        next_stage_points.append((a**2,y))
    return next_stage_points
        

def compute_next_layer(current_stage_point, calculated_points, rand):
    next_stage_points = []
    for point_a, point_neg_a in zip (current_stage_point, calculated_points):
        a,f_a = point_a
        neg_a, neg_f_a = point_neg_a
        y = f_a + (f_a - neg_f_a)/(2*a)*(rand-a)
        next_stage_points.append((a**2,y))
    return next_stage_points

def verify_index_x_value(current_stage_points, indexes, domain_len, field_gen):
    group_gen = field_gen ** ((PRIME-1)/domain_len)
    new_indexes = [reverse_bit(i, log(domain_len,2)) for i in indexes]
    alleged_x = [field_gen*(group_gen**i) for i in new_indexes]
    prover_x = [x[0] for x in current_stage_points]
    assert alleged_x == prover_x